In [ ]:
# ============================================================================
# 导入所有必需的库和模块
# ============================================================================

# 基础库
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# 可视化
import matplotlib.pyplot as plt
import seaborn as sns

# API 和模型
from ambiguity_detection_utils import AmbiguityDetector
from ollama import Client
from prompts import PSEUDOCODE_PROMPT_TEMPLATE

# 深度学习模型
# import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer

# 机器学习评估指标
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import (
    roc_auc_score, roc_curve, f1_score, 
    precision_recall_curve, auc,
    confusion_matrix, classification_report, 
    accuracy_score
)

print("所有库导入完成！")

所有库导入完成！


In [2]:
# ============================================================================
# 创建检测器和加载数据
# ============================================================================

# 创建语义歧义检测器
detector = AmbiguityDetector("semantic")

# 加载并准备数据
file_path = r"data/User Story Ambiguity Dataset_A Comprehensive Research Resource/Cornelius_2025_user_story_ambiguity_dataset.xlsx"
df_test = detector.load_and_prepare_data(file_path, create_balanced=True)

print(f"测试数据集: {len(df_test)} 个用户故事")

Semantic歧义统计:
  有semantic歧义: 2831 (22.0%)
  无semantic歧义: 10016 (78.0%)
平衡测试集: 5662 个用户故事
  HasAmbiguity=True: 2831 (50.0%)
  HasAmbiguity=False: 2831 (50.0%)
测试数据集: 5662 个用户故事


In [3]:
df_test = df_test.head()
df_test

,StoryID,StoryText,HasAmbiguity
0,US-C3-6636,"As a seller, I would like to track order to fi...",False
1,US-C7-12360,"As a store owner, I want to track order in ord...",False
2,US-C2-3691,"As a patient, I need to access records so that...",False
3,US-C5-9039,"As a investor, I want to handle finances in or...",True
4,US-C1-610,"As a financial advisor, I need to manage accou...",True


In [4]:
df_test.count()

StoryID         5
StoryText       5
HasAmbiguity    5
dtype: int64

In [5]:
ollama_client = Client(
    host="http://2yo6159kw603.vicp.fun/",
)

In [6]:
print("加载Jina代码embedding模型...")
code_model_name = "jinaai/jina-code-embeddings-0.5b"

code_embedding_model = SentenceTransformer(
    code_model_name,
    tokenizer_kwargs={"padding_side": "left"},
)

print(f"代码embedding模型已加载")
print(f"模型名称: {code_model_name}")

加载Jina代码embedding模型...
代码embedding模型已加载
模型名称: jinaai/jina-code-embeddings-0.5b


In [7]:
# 定义代码embedding函数
def get_code_embedding(code_text, model):
    """
    获取代码文本的embedding向量（使用SentenceTransformer）
    
    参数:
        code_text: 代码文本
        model: SentenceTransformer embedding模型
    
    返回:
        embedding: 代码embedding向量 (1, embedding_dim)
    """
    # 使用SentenceTransformer的encode方法，指定prompt为nl2code_document
    # 因为我们要编码代码/伪代码样本，不是查询
    embedding = model.encode(
        code_text,
        prompt_name="nl2code_document",
        convert_to_numpy=True
    )
    
    # 如果返回的是1D向量，需要reshape为2D
    if embedding.ndim == 1:
        embedding = embedding.reshape(1, -1)
    
    return embedding

print("get_code_embedding 函数已定义")

get_code_embedding 函数已定义


In [10]:
# 定义计算伪代码相似度分数的函数
def calculate_pseudocode_similarity_score(pseudocodes, model):
    """
    计算伪代码批次的相似度分数
    
    使用与Clarification Score相同的逻辑，但基于代码embedding而非文本embedding
    
    公式: g(P_i^(q)) = ∑_{(p_ij, p_ij') ∈ P_i^(q) × P_i^(q)} (1 - sim(p_ij, p_ij'))
    
    参数:
        pseudocodes: 伪代码列表
        model: SentenceTransformer代码embedding模型
    
    返回:
        score: 伪代码相似度分数（高分表示高歧义）
        similarity_matrix: 相似度矩阵 (n, n)
    """
    n = len(pseudocodes)
    
    if n < 2:
        return 0.0, np.array([[1.0]])
    
    # 1. 批量获取所有伪代码的embedding向量
    embeddings = model.encode(
        pseudocodes,
        prompt_name="nl2code_document",
        convert_to_numpy=True
    )  # 返回 (n, embedding_dim)
    
    # 2. 计算所有伪代码对的相似度矩阵
    similarity_matrix = cosine_similarity(embeddings)  # (n, n)
    
    # 3. 计算分数：所有非对角线相似度倒数和
    score = 0.0
    
    for i in range(n):
        for j in range(n):
            if i != j:
                dissimilarity = 1 - similarity_matrix[i][j]
                score += dissimilarity
    
    return score, similarity_matrix

print("calculate_pseudocode_similarity_score 函数已定义")

calculate_pseudocode_similarity_score 函数已定义


In [11]:
prompt_template = PSEUDOCODE_PROMPT_TEMPLATE

In [12]:
# 14. 生成伪代码 - 为每个需求生成10份伪代码
# 使用前面的df_test数据（前10个故事）
pseudocode_results = []

print("开始生成伪代码...\n")

for index, row in df_test.iterrows():
    story_id = row['StoryID']
    story_text = row['StoryText']
    pseudocodes = []
    
    print(f"处理故事 {index + 1}/{len(df_test)}: {story_id}")
    
    # 为该故事生成10份伪代码
    for attempt in range(10):
        # 使用导入的模板，通过 format() 替换 {REQUIREMENT} 占位符
        formatted_prompt = PSEUDOCODE_PROMPT_TEMPLATE.format(REQUIREMENT=story_text)
        
        try:
            response = ollama_client.chat(model='gpt-oss:20b', messages=[
                {
                    'role': 'user',
                    'content': formatted_prompt,
                },
            ])
            pseudocode = response.message.content
            pseudocodes.append(pseudocode)
            print(f"  生成伪代码 {attempt + 1}/10")
        except Exception as e:
            print(f"  生成伪代码 {attempt + 1}/10 失败: {str(e)}")
            pseudocodes.append(f"Error: {str(e)}")
    
    # 计算该故事的伪代码相似度分数
    if all(not pc.startswith("Error") for pc in pseudocodes):
        try:
            score, similarity_matrix = calculate_pseudocode_similarity_score(
                pseudocodes,
                code_embedding_model
            )
            pseudocode_ambiguity_level = 'High' if score > 5 else ('Medium' if score > 2 else 'Low')
        except Exception as e:
            print(f"  计算相似度分数失败: {str(e)}")
            score = 0.0
            similarity_matrix = np.array([[1.0]])
            pseudocode_ambiguity_level = 'Error'
    else:
        score = 0.0
        similarity_matrix = np.array([[1.0]])
        pseudocode_ambiguity_level = 'Error'
    
    pseudocode_results.append({
        'StoryID': story_id,
        'StoryText': story_text,
        'HasAmbiguity': row['HasAmbiguity'],
        'NumPseudocodes': len(pseudocodes),
        'PseudocodeScore': score,
        'PseudocodeAmbiguityLevel': pseudocode_ambiguity_level,
        'Pseudocodes': pseudocodes
    })
    
    print(f"  相似度分数: {score:.4f}, 歧义级别: {pseudocode_ambiguity_level}\n")

print(f"完成生成 {len(pseudocode_results)} 个故事的伪代码")

开始生成伪代码...

处理故事 1/5: US-C3-6636
  生成伪代码 1/10
  生成伪代码 2/10
  生成伪代码 3/10
  生成伪代码 4/10
  生成伪代码 5/10
  生成伪代码 6/10
  生成伪代码 7/10
  生成伪代码 8/10
  生成伪代码 9/10
  生成伪代码 10/10
  相似度分数: 17.8259, 歧义级别: High

处理故事 2/5: US-C7-12360
  生成伪代码 1/10
  生成伪代码 2/10
  生成伪代码 3/10
  生成伪代码 4/10
  生成伪代码 5/10
  生成伪代码 6/10
  生成伪代码 7/10
  生成伪代码 8/10
  生成伪代码 9/10
  生成伪代码 10/10
  相似度分数: 16.6445, 歧义级别: High

处理故事 3/5: US-C2-3691
  生成伪代码 1/10
  生成伪代码 2/10
  生成伪代码 3/10
  生成伪代码 4/10
  生成伪代码 5/10
  生成伪代码 6/10
  生成伪代码 7/10
  生成伪代码 8/10
  生成伪代码 9/10
  生成伪代码 10/10
  相似度分数: 17.4181, 歧义级别: High

处理故事 4/5: US-C5-9039
  生成伪代码 1/10
  生成伪代码 2/10
  生成伪代码 3/10
  生成伪代码 4/10
  生成伪代码 5/10
  生成伪代码 6/10
  生成伪代码 7/10
  生成伪代码 8/10
  生成伪代码 9/10
  生成伪代码 10/10
  相似度分数: 19.3483, 歧义级别: High

处理故事 5/5: US-C1-610
  生成伪代码 1/10
  生成伪代码 2/10
  生成伪代码 3/10
  生成伪代码 4/10
  生成伪代码 5/10
  生成伪代码 6/10
  生成伪代码 7/10
  生成伪代码 8/10
  生成伪代码 9/10
  生成伪代码 10/10
  相似度分数: 21.0353, 歧义级别: High

完成生成 5 个故事的伪代码


In [13]:
pseudocode_results[0]

{'StoryID': 'US-C3-6636',
 'StoryText': 'As a seller, I would like to track order to find desired items with detailed specifications and comprehensive requirements',
 'HasAmbiguity': False,
 'NumPseudocodes': 10,
 'PseudocodeScore': np.float32(17.825926),
 'PseudocodeAmbiguityLevel': 'High',
 'Pseudocodes': ['```\n// Data structures\nclass Specification {\n    attributes: Map<String, String>   // e.g., "color" -> "red", "size" -> "M"\n}\n\nclass Requirement {\n    description: String\n    priority: Integer\n}\n\nclass Item {\n    id: String\n    name: String\n    specifications: Specification\n    requirements: List<Requirement>\n}\n\nclass Order {\n    orderId: String\n    customerId: String\n    items: List<Item>\n    status: String\n    createdDate: Date\n}\n\n// Repository (in-memory example)\nordersDB: Map<String, Order>  // key = orderId\n\n// Service functions\n\nfunction createOrder(customerId: String, itemList: List<Item>) -> String {\n    orderId = generateUniqueId()\n    ord